In [1]:
import pandas as pd
import json

 ### Data loading

In [ ]:
data = pd.read_excel("../Downloads/LYLA 2022-9-21_latest.xlsx")
data = data[(data["ly_yr"] >=2010) & (data["ly_yr"]<=2019)]

In [22]:
class AttributeDict(dict):
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

### List of special events with their corresponding header

In [23]:
ids = [2002, 715,1357,555,2398,2110,2249,2322,2498,2631,2303]
links = ["https://www.lainformacion.com/mundo/linchan-a-un-alcalde-en-guatemala-acusado-de-atacar-a-su-rival-politico_PBooZmlricaLMs8Wr1LeH/",
"https://www.jornada.com.mx/2004/11/24/042n3cap.php",
"https://cnnespanol.cnn.com/2015/10/21/pobladores-linchan-a-encuestadores-tras-confundirlos-con-secuestradores/",
"https://www.eltiempo.com/justicia/investigacion/la-historia-detras-del-hombre-que-fue-linchado-en-bogota-por-una-falsa-noticia-289508",
"https://www.opinion.com.bo/articulo/policial/matan-pedradas-j-oacute-venes-acusados-cometer-robo/20181121112600633260.html",
"https://www.clarin.com/policiales/linchamiento-rosario-imputados-declaro-culpable-ira-preso_0_JZXHTCR.html",
"https://www.cronica.com.ar/policiales/Presunto-ladron-habria-sido-linchado-por-fieles-que-recordaban-Semana-Santa-20190420-0020.html",
"https://www.abc.es/internacional/20131109/abci-linchamientos-convierten-bolivia-pais-201311081733.html",
"https://www.elcaribe.com.do/sin-categoria/linchamientos-comienzan-ser-comunes-barriadas/",
"https://elcomercio.pe/peru/puno/juliaca-establecido-castigo-popular-324709-noticia/",
"https://www.premiosgoya.com/pelicula/tribus-de-la-inquisicion/"]

headers = ["\"Linchan a un alcalde en Guatemala, acusado de atacar a su rival político\"",
          "Turba quema vivos a dos agentes de la PFP; otro en estado grave",
          "Pobladores linchan a encuestadores tras confundirlos con secuestradores",
          "La verdadera historia del hombre linchado por falsa noticia en redes",
          "Matan a pedradas a dos jóvenes acusados de cometer robo",
          "Linchamiento en Rosario: uno de los tres imputados se declaró culpable y no irá preso",
          "Presunto ladrón habría sido linchado por fieles que recordaban Semana Santa",
          "Los linchamientos convierten a Bolivia en un país con pena de muerte de facto",
          "Linchamientos comienzan a ser comunes en barriadas",
          "En Juliaca se ha establecido el castigo popular",
          "Tribus de la Inquisición"
          ]


id_link_dict = {key:value for (key,value) in zip(ids,links)}

id_header_dict = {key:value for (key,value) in zip(ids,headers)}






In [24]:
id_link_dict

{2002: 'https://www.lainformacion.com/mundo/linchan-a-un-alcalde-en-guatemala-acusado-de-atacar-a-su-rival-politico_PBooZmlricaLMs8Wr1LeH/',
 715: 'https://www.jornada.com.mx/2004/11/24/042n3cap.php',
 1357: 'https://cnnespanol.cnn.com/2015/10/21/pobladores-linchan-a-encuestadores-tras-confundirlos-con-secuestradores/',
 555: 'https://www.eltiempo.com/justicia/investigacion/la-historia-detras-del-hombre-que-fue-linchado-en-bogota-por-una-falsa-noticia-289508',
 2398: 'https://www.opinion.com.bo/articulo/policial/matan-pedradas-j-oacute-venes-acusados-cometer-robo/20181121112600633260.html',
 2110: 'https://www.clarin.com/policiales/linchamiento-rosario-imputados-declaro-culpable-ira-preso_0_JZXHTCR.html',
 2249: 'https://www.cronica.com.ar/policiales/Presunto-ladron-habria-sido-linchado-por-fieles-que-recordaban-Semana-Santa-20190420-0020.html',
 2322: 'https://www.abc.es/internacional/20131109/abci-linchamientos-convierten-bolivia-pais-201311081733.html',
 2498: 'https://www.elcaribe.

**Function for adding new variables needed for plotting events on the map in web app.**

In [25]:

def map_to_obj(i, item):
    if int(item.id) in [2002, 715,1357,555,2398,2110,2249,2322,2498,2631]:
        item.press_article = "true"
        item.link = id_link_dict[int(item.id)]
        item.header = id_header_dict[int(item.id)]
    else:
        item.press_article = "false"
        item.link = "null"
        item.header = "null"
    item.geometry = item.geom[0]
    list_coords = item.geom[0]['coordinates'].replace(" ", "").replace('\'',"").split(',')
    if len(list_coords) > 2:
        list_coords = list_coords[:2]
    item.geometry['coordinates'] = [float(s) for s in list_coords]
    item.pop('geom', None)
    item.date = item.date.strftime('%Y-%m-%d')
    return item


In [26]:
def table_to_json(file: str):
    data = pd.read_excel(file)

    print(f'Reading {len(data)} rows and {len(data.columns)} columns')
    data["type"] = ["Point"]*len(data)

    j = (data.groupby(['id','tar1_sex','name_0', 'name_1', 'tar1_age','pe_violence','st_outcome', 'tar_outcome', 'evidence1_text', 'evidence1_source', 'tar_wrongdoing', 'date'], dropna=False)
           .apply(lambda x: x[['type','coordinates']].to_dict('records'))
           .reset_index()
           .rename(columns={0:'geom'})
           .to_dict(orient='records'))

    dict_data = [AttributeDict(i) for i in j]

    json_data = [x for x in (map_to_obj(v) for v in dict_data) if x]
    return json_data


In [30]:
print(f'Reading {len(data)} rows and {len(data.columns)} columns')
data["type"] = ["Point"]*len(data)
end = pd.to_datetime('2023-12-31')
data["date"] = data["date"].fillna(end)

Reading 2818 rows and 122 columns


In [32]:
j = (data.groupby(['id','name_0', 'name_1', 'tar1_age','tar1_sex','pe_violence','st_outcome', 'tar_outcome', 'evidence1_text', 'evidence1_source', 'tar_wrongdoing','pe_approxnumber', 'date'], dropna=False)
           .apply(lambda x: x[['type','coordinates']].to_dict('records'))
           .reset_index()
           .rename(columns={0:'geom'})
           .to_dict(orient='records'))
dict_data = [AttributeDict(i) for i in j]
dict_data

In [ ]:
json_data = [x for x in (map_to_obj(i, v) for i, v in enumerate(dict_data)) if x]

In [50]:
json_data[:5]

[{'id': 2,
  'name_0': 'Brazil',
  'name_1': 'Rio de Janeiro',
  'tar1_age': 2.0,
  'tar1_sex': 1.0,
  'pe_violence': nan,
  'st_outcome': 1.0,
  'tar_outcome': nan,
  'evidence1_text': 'Policiais do 16o BPM (Olaria) prenderam quatro bandidos, acusados de aterrorizar e fazer um arrastão, na manhã de ontem, na Avenida Meriti, na Vila da Penha. Wesley Estevão, de 23 anos, Carlos de Paula, de 28, e os irmãos Daniel e Mawuell Silva, de 25 e 28. Eles teriam roubado um carro. Os policiais impediram o linchamento de dois deles por parte de moradores.',
  'evidence1_source': 'O Globo',
  'tar_wrongdoing': 1.0,
  'pe_approxnumber': 1.0,
  'date': '2010-01-24',
  'press_article': 'false',
  'link': 'null',
  'header': 'null',
  'geometry': {'type': 'Point', 'coordinates': [-22.843558, -43.313145]}},
 {'id': 4,
  'name_0': 'Brazil',
  'name_1': 'São Paulo',
  'tar1_age': nan,
  'tar1_sex': 1.0,
  'pe_violence': 1.0,
  'st_outcome': 1.0,
  'tar_outcome': 2.0,
  'evidence1_text': ' Alguns da co

In [35]:

with open("events_data.json", "w") as f:
    json.dump(json_data, f, indent=4, ensure_ascii=False)

## Convert original dataset to json file

**This is the dataset that can be downloaded in the web application.**

In [16]:
data_orig = pd.read_excel("../Downloads/LYLA 2022-9-21_latest.xlsx")
data_orig.head()


,coder,id,cc,name_0,name_1,name_local,cve,cve_ent,cve_mun,alcaldia,...,evidence1_day,evidence1_month,evidence1_year,evidence2_text,evidence2_source,evidence2_date,evidence2_day,evidence2_month,evidence2_year,longcodebook
0,AB,654,70,Mexico,Oaxaca,Santiago Astata,20453.0,20.0,453.0,NaN,...,6,1,2000,Después de permanecer atados a un poste durant...,Servicio Universal de noticias,2000-01-07,7.0,1.0,2000.0,1
1,AB,655,70,Mexico,México,Fundidores (mercado 3 de junio),15031.0,15.0,31.0,NaN,...,1,3,2000,NaN,NaN,NaT,NaN,NaN,NaN,1
2,AB,656,70,Mexico,Distrito Federal,Barrio la Joya,9005.0,9.0,5.0,GUSTAVO A. MADERO,...,28,3,2000,NaN,NaN,NaT,NaN,NaN,NaN,1
3,AB,657,70,Mexico,Puebla,Ayoxuxtla,21073.0,21.0,73.0,NaN,...,3,4,2000,Las autoridades tuvieron que intervenir para e...,Servicio Universal de noticias,2000-04-03,3.0,4.0,2000.0,1
4,AB,658,70,Mexico,Chiapas,Las Margaritas,7052.0,7.0,52.0,NaN,...,13,5,2000,NaN,NaN,NaT,NaN,NaN,NaN,1


In [19]:
end = pd.to_datetime('2023-12-31')

data_orig["date"] = data_orig["date"].fillna(end)
data_orig["yearmonth"] = data_orig["yearmonth"].fillna(end)
data_orig["evidence1_date"] = data_orig["evidence1_date"].fillna(end) # events without date are filled with 2023-12-31
data_orig["evidence2_date"] = data_orig["evidence2_date"].fillna(end)

df_dict = (data_orig.to_dict(orient='records'))

In [20]:
df_dict

def map_to_obj(i, item):
    
   
    item["date"] = item["date"].strftime('%Y-%m-%d')
    item["yearmonth"] = item["yearmonth"].strftime('%Y-%m-%d')
    item['evidence1_date'] =item['evidence1_date'].strftime('%Y-%m-%d')
    item['evidence2_date'] =item['evidence2_date'].strftime('%Y-%m-%d')
    return item

df_dict = [map_to_obj(i,item) for i, item in enumerate(df_dict)]

#### Save json file

In [22]:
with open("LYLA_2022-9-21_latest.json", "w") as f:
    json.dump(df_dict, f, indent=4, ensure_ascii=False)